In [61]:
import pandas as pd

df_accidents = pd.read_pickle("accidents.pkl.gz")

# Filter accidents based on road type
df_filtered_accidents = df_accidents[df_accidents["p36"].isin([0, 1])]


In [62]:
contingency_table = pd.crosstab(df_filtered_accidents["p36"], df_filtered_accidents["p9"])

highway_probability = contingency_table.iloc[0,0] / (contingency_table.iloc[0,0] + contingency_table.iloc[0,1])
first_class_probability = contingency_table.iloc[1,0] / (contingency_table.iloc[1,0] + contingency_table.iloc[1,1])

print("Pravdepodobnost nasledkov na zdravi pri nehodach na dialniciach:", highway_probability)
print("Pravdepodobnost nasledkov na zdravi pri nehodach na cestach prvej triedy:",  first_class_probability)

Pravdepodobnost nasledkov na zdravi pri nehodach na dialniciach: 0.15742961747254133
Pravdepodobnost nasledkov na zdravi pri nehodach na cestach prvej triedy: 0.3233327226090143


In [63]:
from scipy.stats import chi2_contingency

# Chi2 test
chi2, p, dof, expected = chi2_contingency(contingency_table)


print("Zaver:")
if p < 0.05:
    print("Na cestach prvej triedy neboli nehody s nasledkami na zdravi s rovnakou pravdepodbnostou ako na dialniciach.")
else:
    print("Na cestach prvej triedy boli nehody s nasledkami na zdravi s rovnakou pravdepodbnostou ako na dialniciach.")

Zaver:
Na cestach prvej triedy neboli nehody s nasledkami na zdravi s rovnakou pravdepodbnostou ako na dialniciach.


# Hypotéza 1: Na silnicích první třídy se byly nehody s následky na zdraví se stejnou pravděpodobností jako na dálnicích.

Pravdepodobnosť, že nehoda na ceste prvej triedy vedie k následkom na zdraví, je **32,33%**.

Pravdepodobnosť, že nehoda na diaľnici vedie k následkom na zdraví, je **15,74%**.

Na analýzu bol použitý **chi-kvadrát test**.

Hypotézu 1 s 95% istotou **zamietame**. Na cestách prvej triedy neboli nehody s následkami na zdraví s rovnakou pravdepodobnosťou ako na diaľniciach.


In [64]:
df_vehicles = pd.read_pickle("vehicles.pkl.gz")

# Merge dataframes on p1 to include financial damage
df_vehicles = pd.merge(df_vehicles, df_accidents[['p1', 'p14*100']], on='p1', how='inner')


# Filter trolleybus and bus accidents, extracting damages
trolleybus_damages = df_vehicles[df_vehicles["p44"] == 11]["p14*100"]
bus_damages = df_vehicles[df_vehicles["p44"] == 8]["p14*100"]


In [65]:
from scipy.stats import shapiro

# Normality tests
_, trolleybus_normality = shapiro(trolleybus_damages)
_, bus_normality = shapiro(bus_damages)

if trolleybus_normality < 0.05:
    print("Trolleybus: not a normal distribution")
if bus_normality < 0.05:
    print("Bus: not a normal distribution")

Trolleybus: not a normal distribution
Bus: not a normal distribution


In [66]:
from scipy.stats import mannwhitneyu

# Mann-Whitney U Test
u_stat, p_value = mannwhitneyu(trolleybus_damages, bus_damages, alternative="less")

print("Zaver:")
if p_value < 0.05:
    print("Skody pri nehodach trolejbusov su statisticky vyznamne nizsie, nez pri nehodach autobusov.")
else:
    print("Skody pri nehodach trolejbusov nie su statisticky vyznamne nizsie, nez pri nehodach autobusov.")


Zaver:
Skody pri nehodach trolejbusov su statisticky vyznamne nizsie, nez pri nehodach autobusov.


# Hypotéza 2: Škoda při nehodách trolejbusů je nižší, než při nehodách autobusů a tato odchylka je statisticky významná

Pre zistenie, či majú dáta normálne rozdelenie, bol použitý **Shapiro-Wilk test**. Výsledná p-hodnota pre autobusy aj trolejbusy bola menšia ako 0,05, čo znamená, že dáta nemajú normálne rozdelenie. Z tohto dôvodu bol použitý **Mann-Whitney U test**.

Na základe výsledkov testu je možné s 95% istotou **potvrdiť** hypotézu, že škody pri nehodách trolejbusov sú štatisticky významne nižšie ako pri nehodách autobusov.
